## Recurrent Neural networks and Keras in Tensorflow
** How too install tensorflow: https://www.tensorflow.org/install/ **

* `pip install tensorflow` -- **cpu-only** version for Linux & Mac OSX
* if you want GPU support try -- `pip install tensorflow-gpu`

For this notebook:
```
conda create -n py36_tensorflow python=3.6 anaconda
source activate py36_tensorflow
pip install tensorflow
```

### About RNNs

Recurrent layers extract features and hidden connections along a sequence.

## Let's look at popular varieties.

#### PureRNN
<img src="img/pure_rnn.png" width="600">


#### LSTM
article - http://www.bioinf.jku.at/publications/older/2604.pdf
<img src="img/lstm.png" width="600">

#### GRU
article - https://arxiv.org/abs/1406.1078
<img src="img/gru.png" width="600">

## Try power of the RNNs

<img src="img/rnns.jpeg" width="600">

#### Karpathy blog:  http://karpathy.github.io/2015/05/21/rnn-effectiveness/
#### Denny Britz blog: http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/
#### Seq2seq example: https://github.com/Scitator/YATS2S/blob/versions/tf_1.2/seq2seq_example_tatoeba_inference.ipynb
#### About Keras: https://habrahabr.ru/company/ods/blog/325432/

## Goal for today:
1. Create RNN-model (many2one), LM like
2. Use it to generate text (logic: many many2one = many2many)

### Colab link: https://colab.research.google.com/drive/1yhe2ZbqMC70rHa8GhaOdopc1wwcgGbtS

In [ ]:
import io
import sys
import random
import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# when keras comes to tensorflow :slava_2:
layers = tf.keras.layers
models = tf.keras.models
optimizers = tf.keras.optimizers
callbacks = tf.keras.callbacks
get_file = tf.keras.utils.get_file

In [ ]:
tf.test.is_gpu_available()

# How does it work?

## 1. We need some corpora...

In [ ]:
# EN
path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt') # amazon!
text = io.open(path, encoding='utf-8').read().lower().replace("\n", "")

In [ ]:
print('corpus length:', len(text))
print('text sample: ', text[:200])

## 2. Data preprocessing - encode all your data to numbers
3 ways:
1. word-level: stemming, lemmatization -> tokenization
    + (+) w2v embeddings can be used (GoogleVec, Glove, etc)
    - (-/+) a bit complex preprocessing
    - (-) OOV words
2. char-level: tokenize them all!
    - (+) quite simple
    - (+) no OOV
    - (-/+) complex model
3. subword-level: ngrams or BPE
    - (+) no OOV (only sometimes)
    - (-/+) complex preprocessing
    - (-/+) complex model
    - pretrained w2v - https://github.com/bheinzerling/bpemb

### 2.1 Vocabulary creation

In [ ]:
# as we are lazy enoght, let's use char level
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

### 2.2 Model input creation - sequences of same len

In [ ]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

## 2.3 Finally, vectorization

In [ ]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

## 3. Now, let's create simple model

In [ ]:
model = models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars)))
model.add(layers.Activation('softmax'))

optimizer = optimizers.RMSprop(lr=0.01)
model.compile(
    loss='categorical_crossentropy', 
    optimizer=optimizer)

In [ ]:
model.summary()

In [ ]:
# If you have graphviz and pydot installed correctly.
# Beautifull Keras-model visualization.
# tf.keras.utils.plot_model(model)

### 3.1 Our sample method
Takes logits and outputs 'most probable' variant

About temperature: https://cs.stackexchange.com/questions/79241/what-is-temperature-in-lstm-and-neural-networks-generally

In [ ]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

### 3.1 Training callback

Takes current model and generates different texts with it.

In [ ]:
def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [ ]:
print_callback = callbacks.LambdaCallback(on_epoch_end=on_epoch_end)

## 4. Train them all!

In [ ]:
history = model.fit(
    x, y,
    batch_size=128,
    epochs=5,
    callbacks=[print_callback])

---

## 5. Visualization stuff

In [ ]:
import os

import matplotlib

matplotlib.use('Agg')
import matplotlib.pyplot as plt  # noqa: E402

plt.style.use("ggplot")
import seaborn as sns  # noqa: E402

sns.set(color_codes=True)


def create_if_need(path):
    if not os.path.exists(path):
        os.makedirs(path)


def plot_rutine(
        history, metric,
        legend=None, additional_info=None,
        show=False, save_dir=None,
        last_steps=None, last_steps_reference=None):
    title = 'model {}'.format(metric)
    if additional_info is not None:
        title += " {}".format(additional_info)
    plt.title(title)
    plt.ylabel(metric)
    plt.xlabel('epoch')

    if save_dir is not None:
        filename = "{}/{}".format(save_dir, metric)
        if additional_info is not None:
            filename += "_{}".format(str(additional_info))
        filename += ".png"
        if legend is not None:
            plt.savefig(filename, format='png', dpi=300,
                        bbox_extra_artists=(legend,), bbox_inches='tight')
        else:
            plt.savefig(filename, format='png', dpi=300)
    if show:
        plt.show()

    if last_steps is not None and isinstance(last_steps, float):
        last_steps = int(last_steps * len(history[metric]))

    if last_steps is not None and isinstance(last_steps, int) and last_steps_reference is not None:
        last_steps_history = {metric: history[metric][-last_steps:]}
        val_metric = "val_{}".format(metric)
        if val_metric in history.keys():
            last_steps_history[val_metric] = history[val_metric][-last_steps:]
        last_steps_reference(
            history=last_steps_history,
            metric=metric,
            additional_info="last_steps",
            show=show,
            save_dir=save_dir)


def plot_bimetric(
        history, metric,
        additional_info=None,
        show=False, save_dir=None, last_steps=None):
    plt.figure()

    if "log_epochs" in history:
        plt.plot(history["log_epochs"], history[metric])
        plt.plot(history["log_epochs"], history['val_{}'.format(metric)])
    else:
        plt.plot(history[metric])
        plt.plot(history['val_{}'.format(metric)])

    lgn = plt.legend(['train', 'val'], loc='center left',
                     bbox_to_anchor=(1, 0.5))

    plot_rutine(
        history, metric,
        legend=lgn, additional_info=additional_info,
        show=show, save_dir=save_dir,
        last_steps=last_steps, last_steps_reference=plot_bimetric)


def plot_unimetric(
        history, metric,
        additional_info=None,
        show=False, save_dir=None, last_steps=None):
    plt.figure()

    if "log_epochs" in history:
        plt.plot(history["log_epochs"], history[metric])
    else:
        plt.plot(history[metric])

    plot_rutine(
        history, metric,
        legend=None, additional_info=additional_info,
        show=show, save_dir=save_dir,
        last_steps=last_steps, last_steps_reference=plot_unimetric)


def plot_all_metrics(
        history,
        show=False, save_dir=None, last_steps=None):
    if save_dir is not None:
        create_if_need(save_dir)

    bimetrics = []
    for metric in history.keys():
        if metric.startswith("val_"):
            bimetrics.append(metric[4:])

    for metric in history.keys():
        if not metric.startswith("val_"):
            if metric in bimetrics:
                plot_bimetric(
                    history, metric,
                    additional_info=None, show=show,
                    save_dir=save_dir, last_steps=last_steps)
            else:
                plot_unimetric(
                    history, metric,
                    additional_info=None, show=show,
                    save_dir=save_dir, last_steps=last_steps)

In [ ]:
%matplotlib inline

In [ ]:
plot_all_metrics(history.history, show=True)